# Exercise 1.2

## Overview

The solution to the 2nd task of Exercise 1 focuses on counting the number of cars travelling towards the `City Centre` in each of the provided videos using similar frame differencing techniques used in the 1st task of this exercise. This notebook implements the solution and explains the step-by-step implementation process. The theoretical aspects of these techniques will be further elaborated in the report submitted alongside this notebook.

## Implementation

### Installing Dependencies

In [1]:
%pip install opencv-python numpy pandas

Note: you may need to restart the kernel to use updated packages.


### Count cars heading towards City Centre  

To begin, we first setup an array to hold the paths of the videos in which the cars should be counted.

In [2]:
video_paths = ['videos/Traffic_Laramie_1.mp4', 'videos/Traffic_Laramie_2.mp4']

We'll be using very similar logic to the first task but with some additions to process. Since we have two videos, we'll move the logic to function to make it reusable. 

The below function will count the number of cars passing in a direction within the detection bounding box and it will follow the following steps:

1. **Loading Video**: To begin, we first load the video using the `load_video_file` function from the `cv_utils.py` file, which internally uses the python opencv library. 

2. **Calculate Background Frame**: The background frame is obtained by calculating the median frame from a sample of 200 random frames the video. This is done by the `get_median_frame` in the `cv_utils.py` file.

3. **Reading Frames**: Then each frame of the video is read until the end or until the 'q' key is pressed.

4. **Grayscale Conversion**: A grayscale version of the current frame is created using the `cv2.cvtColor()` function. Grayscale images simplify the analysis by removing color information.

5. **Background Subtraction**: The background frame is subtracted from the current grayscale frame to isolate moving objects using the `subtract_background()` function from `cv_utils`.

6. **Preprocessing the Subtracted Frame**: The subtracted frame is preprocessed to enhance the moving objects:
   * **Thresholding**: A threshold is applied to the foreground mask to highlight moving objects.
   * **Dilation**: The shapes in the foreground mask are dilated to fill out black regions within the shapes.
   * **Erosion**: The boundary of the shapes in the foreground mask is eroded to smoothen out the edges of moving objects.

7. **Car Identification**: Bounding boxes of cars are detected using the `cv2.findContours()` function on the preprocessed foreground mask. The cars are identified by comparing the bounding box centroids of current frame with the bounding box centroids of the previous frame and match them if the distance between them in the consecutive frames is within a threshold.

8. **Counting Cars**: The cars that are within the detection area and are heading in the specified direction are counted. Each uniquely identified car is counted only once.

9. **Drawing Bounding Boxes and Counter**: This part is executed only if the `render_window` parameter for the function is `True`. The bounding boxes for the area of detection and cars and the car counter are drawn to a copy of the original frame.

10. **Rendering and Playback**: Similar to the previous step, this step is only executed if the `render_window` parameter for the function is `True`. The modified frame with the bounding boxes and the counter is rendered in a window. The frames are updated based on the frame time of the video to maintain proper playback.

11. **Return Cars Count and Cars Passing per Minute**: Calculate cars count and cars per minute and returning the values as a tuple.

In [3]:
from vector import Vector
from rect import Rect
import cv_utils
import cv2
import numpy as np
from car import Car
from vector_moving_average_filter import VectorMovingAverageFilter

def count_cars_in_video_heading_in_direction(video_path: str, 
                                             direction: Vector, 
                                             area_of_detection_bounding_box: Rect, 
                                             min_car_bounding_box_area: int,
                                             car_tracking_identity_distance_threshold: int=80,
                                             car_direction_angle_threshold: int=20,
                                             render_window: bool=True):
    # Loading video
    video, video_metadata = cv_utils.load_video_file(video_path)
    # Extracting background frame
    background_frame = cv_utils.get_median_frame(video_path, 200)
    background_gray_frame = cv2.cvtColor(background_frame, cv2.COLOR_BGR2GRAY)

    frame_time_ms = int(round(1000 / video_metadata["frame_rate"]))
    
    # Setting up variables for tracking cars
    tracked_cars = {}
    tracked_cars_moving_average_filters = {}
    tracked_cars_absent_frame_counts = {}
    car_id_counter = 0
    counted_car_ids = []
    cars_count = 0
    
    # Read until video is completed or we press 'q'
    while True:
        # Reading frame
        check, frame = video.read()

        if check == True:
            # Creating grayscale version of frame
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Subtracting background frame from current frame
            foreground_frame = cv_utils.subtract_background(background_gray_frame, gray_frame)

            ## Preprocess subtracted frame
            # Thresholding the foreground_mask to highlight the moving objects
            _, foreground_mask = cv2.threshold(foreground_frame, 25, 255, cv2.THRESH_BINARY)
            # Dilate the shapes in the foreground mask to fill out black regions within the shape
            foreground_mask = cv2.dilate(foreground_mask, np.ones((9, 9), np.uint8), iterations=2)
            # Erode the boundary of the shapes in the foreground mask to smoothen out the edges of the moving objects
            foreground_mask = cv2.erode(foreground_mask, np.ones((5, 5), np.uint8), iterations=1)

            # Detect bounding boxes of cars from foreground_mask using contours method from opencv
            car_bounding_boxes = []
            contours, _ = cv2.findContours(foreground_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for contour in contours:
                # Accessing the x, y and height, width of the cars
                x, y, width, height = cv2.boundingRect(contour)
                contourBoundingRect = Rect(Vector(x, y), width, height)

                if cv2.contourArea(contour) > min_car_bounding_box_area and Rect.is_rect_within_rect(area_of_detection_bounding_box, contourBoundingRect):
                    # Recording bounding boxes of the cars
                    car_bounding_boxes.append(contourBoundingRect)

            # Loop through the detected boxes and associating them with existing cars based on new positions or create new ones
            unmatched_tracked_car_ids = set(tracked_cars.keys())
            for car_bounding_box in car_bounding_boxes:
                centroid = car_bounding_box.get_centroid()
                matched_car_id = None
                for car_id, car in tracked_cars.items():
                    distance = Vector.distance(centroid, car.position)
                    if distance < car_tracking_identity_distance_threshold:
                        matched_car_id = car_id
                        break

                if matched_car_id is None:
                    # Create a new car ID
                    car_id = car_id_counter
                    car_id_counter += 1
                    tracked_cars_moving_average_filters[car_id] = VectorMovingAverageFilter(10)
                    smoothed_centroid = tracked_cars_moving_average_filters[car_id].smoothen_value(centroid)
                    tracked_cars[car_id] = Car(smoothed_centroid, Vector(0, 0))
                else:
                    # Smoothening centroid using corresponding moving average filter 
                    smoothed_centroid = tracked_cars_moving_average_filters[matched_car_id].smoothen_value(centroid)
                    # Updating the matched car with new position if the new position is different
                    tracked_cars[matched_car_id].direction = smoothed_centroid - tracked_cars[matched_car_id].position
                    tracked_cars[matched_car_id].position = smoothed_centroid
                    # Removing matched id from set of unmatched car ids
                    unmatched_tracked_car_ids.remove(matched_car_id)

            # Check if cars are no longer available and remove them from tracked_cars dictionary
            for car_id in unmatched_tracked_car_ids:
                # Car not detected in current frame, increment absent frame count
                tracked_cars_absent_frame_counts[car_id] = tracked_cars_absent_frame_counts.get(car_id, 0) + 1

                # Check if car has been absent for too many frames
                if tracked_cars_absent_frame_counts[car_id] > 10:
                    # Remove car from tracked_cars and absent_frame_counts dictionaries
                    del tracked_cars[car_id]
                    del tracked_cars_moving_average_filters[car_id]
                    del tracked_cars_absent_frame_counts[car_id]

            # Checking if any new cars are heading from downtown to city centre and counting them
            for car_id, car in tracked_cars.items():
                if car_id not in counted_car_ids:
                    # Checking if the car is heading in the general direction of the city centre
                    if not car.direction.is_zero() and Vector.angle_between(car.direction, direction) <= car_direction_angle_threshold:
                        cars_count += 1
                        counted_car_ids.append(car_id)
            
            if render_window:
                # Draw bounding box for area of detection
                cv_utils.draw_rect_in_frame(frame, area_of_detection_bounding_box, (0, 0, 255))
                # Drawing bounding box for all the cars
                for car_bounding_box in car_bounding_boxes:
                    cv_utils.draw_rect_in_frame(frame, car_bounding_box, (0, 255, 0))
                # Draw count of cars heading to city centre
                cv_utils.draw_text_in_frame(frame, 
                                            f"Cars Headed to City Centre: {cars_count}",
                                            Vector(30, 60),
                                            (0, 255, 0),
                                            font_scale=0.7)
            
                cv2.imshow("movie", frame)
                
                if cv2.waitKey(frame_time_ms) & 0xFF == ord("q"):
                    break
            else:
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
        else:
            break

    # Release the video object
    video.release()
    
    # Destroy all the windows if rendered
    if render_window:
        cv2.destroyAllWindows()
        
        # Waiting for the windows to close properly
        cv2.waitKey(10)
        
    cars_per_minute = cars_count / (video_metadata['duration_secs']/60)
    
    return cars_count, cars_per_minute
    

With the function for counting the cars heading in a direction ready, we can now prepare the parameters we will be using as arguments when calling the function. The below code snippet intializes the area of detection, which in this case is just junction in the footage, the minimum car bounding box area threshold and the direction to track.

In [4]:
# Creating detection bounding box
area_of_detection_bounding_box = Rect(
    position=Vector(650, 320),
    width=330,
    height=278
)

# Defining minimum size of the calculated bounding boxes to be considered a car
min_car_bounding_box_area = 4000

# Defining direction to track
direction_to_track = Vector(-1, 0)

We then use the initialization variables created above to count cars in both videos provided and display the results as a pandas dataframe. 

In [5]:
import pandas as pd

car_counts = pd.DataFrame()

for video_path in video_paths:
    cars_count, cars_per_minute = count_cars_in_video_heading_in_direction(video_path,
                                                                           direction_to_track,
                                                                           area_of_detection_bounding_box, 
                                                                           min_car_bounding_box_area,
                                                                           render_window=True)
    new_row = {
        'file_name': video_path, 
        'total_cars': cars_count, 
        'cars_per_minute': cars_per_minute
    }
    car_counts = pd.concat([car_counts, pd.DataFrame([new_row])], ignore_index=True)
    
car_counts

,file_name,total_cars,cars_per_minute
0,videos/Traffic_Laramie_1.mp4,6,2.022472
1,videos/Traffic_Laramie_2.mp4,4,2.264151
